In [34]:
import pandas as pd

In [35]:
customers = pd.read_csv('olist_customers_dataset.csv')
order_items = pd.read_csv('olist_order_items_dataset.csv')
orders = pd.read_csv('olist_orders_dataset.csv')

In [36]:

orders.order_purchase_timestamp = pd.to_datetime(orders.order_purchase_timestamp)
orders.order_delivered_customer_date = pd.to_datetime(orders.order_delivered_customer_date)
orders.order_estimated_delivery_date = pd.to_datetime(orders.order_estimated_delivery_date)
orders.order_approved_at = pd.to_datetime(orders.order_approved_at)
orders.order_delivered_carrier_date = pd.to_datetime(orders.order_delivered_carrier_date)

1. Сколько у нас пользователей, которые совершили покупку только один раз? (7 баллов) 

In [37]:
result = orders.merge(customers, on='customer_id', how='outer').query("order_status != 'canceled'").groupby('customer_unique_id', as_index=False).agg({'order_id': 'count'}).query('order_id == 1')
result

,customer_unique_id,order_id
0,0000366f3b9a7992bf8c76cfdf3221e2,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1
2,0000f46a3911fa3c0805444483337064,1
3,0000f6ccb0745a6a4b88665a16c9f078,1
4,0004aac84e0df4da2b147fca70cf8255,1
...,...,...
95555,fffcf5a5ff07b0908bd4e2dbc735a684,1
95556,fffea47cd6d3cc0a88bd621562a9d061,1
95557,ffff371b4d645b6ecea244b27531430a,1
95558,ffff5962728ec6157033ef9805bacc48,1


2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)? (10 баллов)

In [38]:
notdelivered = orders.query("order_status != 'delivered'")
notdelivered = notdelivered.copy()
notdelivered['year'] = notdelivered['order_purchase_timestamp'].dt.year
notdelivered['month'] = notdelivered['order_purchase_timestamp'].dt.month
notdelivered = notdelivered.groupby(['order_status', 'year', 'month'], as_index=False).size().sort_values(['year', 'month'])
notdelivered

,order_status,year,month,size
2,canceled,2016,9,2
69,shipped,2016,9,1
3,canceled,2016,10,24
29,invoiced,2016,10,18
50,processing,2016,10,2
...,...,...,...,...
90,shipped,2018,8,47
112,unavailable,2018,8,7
24,canceled,2018,9,15
91,shipped,2018,9,1


In [39]:
pivot_data = notdelivered.pivot(index=['year', 'month'], columns='order_status', values='size').fillna(0)
pivot_data

order_status  approved  canceled  created  invoiced  processing  shipped  \
year month                                                                 
2016 9             0.0       2.0      0.0       0.0         0.0      1.0   
     10            0.0      24.0      0.0      18.0         2.0      8.0   
2017 1             0.0       3.0      0.0      12.0         9.0     16.0   
     2             1.0      17.0      0.0      11.0        32.0     21.0   
     3             0.0      33.0      0.0       3.0        23.0     45.0   
     4             1.0      18.0      0.0      14.0        10.0     49.0   
     5             0.0      29.0      0.0      16.0        23.0     55.0   
     6             0.0      16.0      0.0      11.0        12.0     47.0   
     7             0.0      28.0      0.0       7.0        11.0     56.0   
     8             0.0      27.0      0.0      20.0        18.0     41.0   
     9             0.0      20.0      0.0      17.0        22.0     38.0   
     10            0.0      26.0      0.0      16.0        20.0     33.0   
     11            0.0      37.0      2.0      35.0        25.0     72.0   
     12            0.0      11.0      2.0      13.0        35.0     57.0   
2018 1             0.0      34.0      0.0      15.0        29.0     74.0   
     2             0.0      73.0      1.0       6.0         6.0     57.0   
     3             0.0      26.0      0.0      23.0         9.0    133.0   
     4             0.0      15.0      0.0      14.0         8.0     99.0   
     5             0.0      24.0      0.0      24.0         6.0     54.0   
     6             0.0      18.0      0.0       3.0         0.0     43.0   
     7             0.0      41.0      0.0      13.0         1.0     60.0   
     8             0.0      84.0      0.0      23.0         0.0     47.0   
     9             0.0      15.0      0.0       0.0         0.0      1.0   
     10            0.0       4.0      0.0       0.0         0.0      0.0   

order_status  unavailable  
year month                 
2016 9                0.0  
     10               7.0  
2017 1               10.0  
     2               45.0  
     3               32.0  
     4                9.0  
     5               31.0  
     6               24.0  
     7               52.0  
     8               32.0  
     9               38.0  
     10              58.0  
     11              84.0  
     12              42.0  
2018 1               48.0  
     2               30.0  
     3               17.0  
     4                5.0  
     5               16.0  
     6                4.0  
     7               18.0  
     8                7.0  
     9                0.0  
     10               0.0

3. По каждому товару определить, в какой день недели товар чаще всего покупается. (7 баллов)

In [40]:
order_week = orders.merge(order_items, on='order_id', how='outer').query("order_status != 'canceled'")
order_week['weekday'] = order_week['order_purchase_timestamp'].dt.day_name()
order_week = order_week.groupby(['weekday', 'product_id'], as_index=False).agg({'order_id':'count'})

In [41]:
most_week = order_week.sort_values("order_id", ascending = False).drop_duplicates(subset = "product_id")
most_week

,weekday,product_id,order_id
53892,Wednesday,422879e10f46682990de24d770e7f83d,93
14460,Monday,99a4788cb24856965c36a24e339b6058,92
39142,Thursday,aca2eb7d00ea1a7b8ebd4e68314663af,89
45142,Tuesday,53b36df67ebb7c41585e8d54d6772e08,76
1889,Friday,368c6c730842d78016ad823897a372db,67
...,...,...,...
23854,Saturday,c73c029653f68c6a83cb371dd148adbd,1
23859,Saturday,c76a7f2aa3596ac11fa33fb6f98cc7c7,1
23866,Saturday,c785fffd281ac9afffc38c8015455fd8,1
23868,Saturday,c7b28fe2ce0fe3c1e58fdb5ccfce9079,1


4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. (8 баллов) 

In [42]:
orders_per_week = customers.merge(orders, on='customer_id', how='outer')
orders_per_week['weekday'] = orders_per_week['order_purchase_timestamp'].dt.isocalendar().week
orders_per_week = orders_per_week.groupby(['customer_unique_id', 'weekday'], as_index=False).agg({'order_id': 'count'}).sort_values('order_id', ascending=False)
orders_per_week

,customer_unique_id,weekday,order_id
7341,12f5d6e1cbf93dafd9dcc19095df0b3d,1,6
14527,25a560b9a6006157838aab1bdbd68624,14,4
50771,83e7958a94bd7f74a9414d8782f87628,1,4
69611,b4e4f24de1e8725b74e4a1f4975116ed,8,4
77026,c8460e4251689ba205045f3ea17884a1,32,4
...,...,...,...
32927,55845d91497951227fd70e450ffb35e0,6,1
32926,55840dd9a1ca54d819ce02d06bacde82,20,1
32925,55836600036b58714e3e52f3921fc36d,19,1
32924,558308f294848bcacb6097d28a2837d4,26,1


5.1. Выполните когортный анализ пользователей. В период с января по декабрь выявите когорту с самым высоким retention на 3-й месяц. Описание подхода можно найти тут. Для визуализации когортной таблицы рекомендуем использовать пример из 8-го урока python, раздел “Стильный урок”, степ 5. (15 баллов)

In [51]:
merged_data = pd.merge(order_items, orders, on='order_id')
merged_data = pd.merge(merged_data, customers, on='customer_id')
merged_data

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18,0c9aeda10a71f369396d0c04dce13a64,65077,sao luis,MA
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,84c5d4fbaf120aae381fad077416eaa0,delivered,2018-07-14 10:26:46,2018-07-17 04:31:48,2018-07-17 08:05:00,2018-07-23 20:31:55,2018-08-01,0da9fe112eae0c74d3ba1fe16de0988b,81690,curitiba,PR
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,29309aa813182aaddc9b259e31b870e6,delivered,2017-10-23 17:07:56,2017-10-24 17:14:25,2017-10-26 15:13:14,2017-10-28 12:22:22,2017-11-10,cd79b407828f02fdbba457111c38e4c4,4039,sao paulo,SP
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,b5e6afd5a41800fdf401e0272ca74655,delivered,2017-08-14 23:02:59,2017-08-15 00:04:32,2017-08-15 19:02:53,2017-08-16 21:59:40,2017-08-25,eb803377c9315b564bdedad672039306,13289,vinhedo,SP


In [52]:
merged_data['order_purchase_timestamp'] = pd.to_datetime(merged_data['order_purchase_timestamp'])
merged_data

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18,0c9aeda10a71f369396d0c04dce13a64,65077,sao luis,MA
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,84c5d4fbaf120aae381fad077416eaa0,delivered,2018-07-14 10:26:46,2018-07-17 04:31:48,2018-07-17 08:05:00,2018-07-23 20:31:55,2018-08-01,0da9fe112eae0c74d3ba1fe16de0988b,81690,curitiba,PR
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,29309aa813182aaddc9b259e31b870e6,delivered,2017-10-23 17:07:56,2017-10-24 17:14:25,2017-10-26 15:13:14,2017-10-28 12:22:22,2017-11-10,cd79b407828f02fdbba457111c38e4c4,4039,sao paulo,SP
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,b5e6afd5a41800fdf401e0272ca74655,delivered,2017-08-14 23:02:59,2017-08-15 00:04:32,2017-08-15 19:02:53,2017-08-16 21:59:40,2017-08-25,eb803377c9315b564bdedad672039306,13289,vinhedo,SP


In [53]:
merged_data['order_month'] = merged_data['order_purchase_timestamp'].dt.to_period('M')
merged_data['cohort_month'] = merged_data.groupby('customer_unique_id')['order_month'].transform('min')
merged_data

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_month,cohort_month
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,2017-09-13 09:45:35,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,2017-09,2017-09
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,2017-04-26 11:05:13,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,2017-04,2017-04
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,2018-01-14 14:48:30,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,2018-01,2018-01
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,2018-08-08 10:10:18,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,2018-08,2018-08
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,2017-02-04 14:10:13,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP,2017-02,2017-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,2018-04-25 04:11:01,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18,0c9aeda10a71f369396d0c04dce13a64,65077,sao luis,MA,2018-04,2018-04
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,84c5d4fbaf120aae381fad077416eaa0,delivered,2018-07-14 10:26:46,2018-07-17 04:31:48,2018-07-17 08:05:00,2018-07-23 20:31:55,2018-08-01,0da9fe112eae0c74d3ba1fe16de0988b,81690,curitiba,PR,2018-07,2018-07
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,29309aa813182aaddc9b259e31b870e6,delivered,2017-10-23 17:07:56,2017-10-24 17:14:25,2017-10-26 15:13:14,2017-10-28 12:22:22,2017-11-10,cd79b407828f02fdbba457111c38e4c4,4039,sao paulo,SP,2017-10,2017-10
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,b5e6afd5a41800fdf401e0272ca74655,delivered,2017-08-14 23:02:59,2017-08-15 00:04:32,2017-08-15 19:02:53,2017-08-16 21:59:40,2017-08-25,eb803377c9315b564bdedad672039306,13289,vinhedo,SP,2017-08,2017-08


In [84]:
merged_data['cohort_lifetime'] = (merged_data['order_month'].dt.year - merged_data['cohort_month'].dt.year) * 12 + (merged_data['order_month'].dt.month - merged_data['cohort_month'].dt.month) + 1
merged_data

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_status,order_purchase_timestamp,...,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_month,cohort_month,cohort_lifetime
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,delivered,2017-09-13 08:59:02,...,2017-09-19 18:34:16,2017-09-20 23:43:48,2017-09-29,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ,2017-09,2017-09,1
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2017-04-26 10:53:06,...,2017-05-04 14:35:00,2017-05-12 16:04:24,2017-05-15,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP,2017-04,2017-04,1
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,delivered,2018-01-14 14:33:31,...,2018-01-16 12:36:48,2018-01-22 13:19:16,2018-02-05,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG,2018-01,2018-01,1
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,delivered,2018-08-08 10:00:35,...,2018-08-10 13:28:00,2018-08-14 13:32:39,2018-08-20,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP,2018-08,2018-08,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,delivered,2017-02-04 13:57:51,...,2017-02-16 09:46:09,2017-03-01 16:42:31,2017-03-17,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP,2017-02,2017-02,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,b51593916b4b8e0d6f66f2ae24f2673d,delivered,2018-04-23 13:57:06,...,2018-04-25 12:09:00,2018-05-10 22:56:40,2018-05-18,0c9aeda10a71f369396d0c04dce13a64,65077,sao luis,MA,2018-04,2018-04,1
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,84c5d4fbaf120aae381fad077416eaa0,delivered,2018-07-14 10:26:46,...,2018-07-17 08:05:00,2018-07-23 20:31:55,2018-08-01,0da9fe112eae0c74d3ba1fe16de0988b,81690,curitiba,PR,2018-07,2018-07,1
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,29309aa813182aaddc9b259e31b870e6,delivered,2017-10-23 17:07:56,...,2017-10-26 15:13:14,2017-10-28 12:22:22,2017-11-10,cd79b407828f02fdbba457111c38e4c4,4039,sao paulo,SP,2017-10,2017-10,1
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,b5e6afd5a41800fdf401e0272ca74655,delivered,2017-08-14 23:02:59,...,2017-08-15 19:02:53,2017-08-16 21:59:40,2017-08-25,eb803377c9315b564bdedad672039306,13289,vinhedo,SP,2017-08,2017-08,1


In [85]:
cohort_counts = merged_data.groupby(['cohort_month', 'cohort_lifetime']).agg({'customer_unique_id' : 'nunique'}).reset_index()
cohort_counts

,cohort_month,cohort_lifetime,customer_unique_id
0,2016-09,1,3
1,2016-10,1,305
2,2016-10,7,1
3,2016-10,10,1
4,2016-10,12,1
...,...,...,...
215,2018-06,3,16
216,2018-07,1,6054
217,2018-07,2,31
218,2018-08,1,6238


In [89]:
initial_customer_count = cohort_counts[cohort_counts['cohort_lifetime'] == 1][['cohort_month', 'customer_unique_id']]
initial_customer_count.rename(columns={'customer_unique_id' : 'cohort_customer_count'}, inplace=True)
initial_customer_count

,cohort_month,cohort_customer_count
0,2016-09,3
1,2016-10,305
10,2016-12,1
12,2017-01,754
30,2017-02,1705
48,2017-03,2595
66,2017-04,2339
83,2017-05,3560
99,2017-06,3114
114,2017-07,3843


In [90]:
cohort_counts = pd.merge(cohort_counts, initial_customer_count, on='cohort_month')
cohort_counts

,cohort_month,cohort_lifetime,customer_unique_id,cohort_customer_count
0,2016-09,1,3,3
1,2016-10,1,305,305
2,2016-10,7,1,305
3,2016-10,10,1,305
4,2016-10,12,1,305
...,...,...,...,...
215,2018-06,3,16,5934
216,2018-07,1,6054,6054
217,2018-07,2,31,6054
218,2018-08,1,6238,6238


In [91]:
cohort_counts['retention'] = cohort_counts['customer_unique_id'] / cohort_counts['cohort_customer_count']
cohort_counts

,cohort_month,cohort_lifetime,customer_unique_id,cohort_customer_count,retention
0,2016-09,1,3,3,1.000000
1,2016-10,1,305,305,1.000000
2,2016-10,7,1,305,0.003279
3,2016-10,10,1,305,0.003279
4,2016-10,12,1,305,0.003279
...,...,...,...,...,...
215,2018-06,3,16,5934,0.002696
216,2018-07,1,6054,6054,1.000000
217,2018-07,2,31,6054,0.005121
218,2018-08,1,6238,6238,1.000000


5.2. В период с января по декабрь выявите когорту с самым высоким retention на 3-й месяц.

In [94]:
highest_retention_cohort = cohort_counts[cohort_counts['cohort_lifetime'] == 3].sort_values(by='retention', ascending=False).iloc[0]
highest_retention_cohort

cohort_month              2017-09
cohort_lifetime                 3
customer_unique_id             22
cohort_customer_count        4090
retention                0.005379
Name: 143, dtype: object